In [1]:
import os

import numpy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cuml.dask.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import joblib

current_path = os.path.join(os.getcwd())
print(current_path)
label_encoder = LabelEncoder()

/media/maro/Mom0-0/Code/final-projects/Pred-Sus-Act


## Loading Data

In [2]:
selected_features_path = os.path.join(
    current_path,
    "data",
    "processed",
    "selected_features.json"
)
selected_features = pd.read_json(selected_features_path)

In [3]:
df = pd.read_csv(os.path.join(current_path,"data", "raw", "train.csv"))
# sampled_df = df#.groupby("label").sample(frac=0.20, random_state=42) # 20% per class
# sampled_df= sampled_df[selected_features]
df.shape

(938583, 22)

In [4]:
X = df.drop(columns=['label'])
y = df['label']
y = label_encoder.fit_transform(y)

# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### USER defined function

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

def generate_metric_report(y_true:np.array, y_pred:np.array, db_save:bool=False)->pd.DataFrame:
    # Using sklearn's classification report, then converting it to dataframe
    report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True))

    report = report.iloc[:-1, :] # excluding last row
    report = report * 100 # convert into percentage

    # Rounding to 2 decimals for each col
    for i, col in enumerate(report.columns):
        report.iloc[:, i] = report.iloc[:, i].apply(lambda v: round(v, 2))
    report.columns = list(label_encoder.inverse_transform(range(0, 6,1))) + ['accuracy', 'macro avg', 'weighted avg']
    return report

def generate_confusion_matrix(y_true:np.array, y_pred:np.array)->pd.DataFrame:
    d= pd.DataFrame(confusion_matrix(y_true, y_pred))
    d.index = list(label_encoder.inverse_transform(range(0, 6,1)))
    d.columns = list(label_encoder.inverse_transform(range(0, 6,1)))
    # class_dist = np.unique(y, return_counts=True)[1]
    # for row in range(len(d.columns)):
    #     d.iloc[row, :] = (d.iloc[row, :]/class_dist[row] *100).round(2)
    return d

# Core Models

## SVM

In [6]:
from cuml.svm import SVC
clf_svc = SVC(kernel="rbf")
clf_svc.fit(x_train, y_train)

SVC()

In [7]:
svc_pred = clf_svc.predict(x_test)
generate_metric_report(y_test, svc_pred)

/home/maro/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maro/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maro/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

,0,1,2,3,4,5,accuracy,macro avg,weighted avg
precision,67.04,73.80,66.07,67.81,70.62,0.0,72.94,57.56,70.89
recall,45.61,98.26,22.93,7.64,31.47,0.0,72.94,34.32,72.94
f1-score,54.28,84.29,34.05,13.73,43.54,0.0,72.94,38.32,67.57


## KNN

In [6]:
from cuml.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors=11)
clf_knn.fit(x_train, y_train)

KNeighborsClassifier()

In [7]:
knn_pred = clf_knn.predict(x_test)
generate_metric_report(y_test, knn_pred)

,BenignTraffic,DDoS,DoS,MITM,Mirai,Recon,accuracy,macro avg,weighted avg
precision,64.11,90.49,86.41,49.73,91.08,60.19,88.58,73.67,88.14
recall,74.63,96.70,70.50,13.97,94.97,30.33,88.58,63.52,88.58
f1-score,68.97,93.49,77.65,21.81,92.98,40.34,88.58,65.87,87.96


In [43]:
generate_confusion_matrix(y_test, knn_pred)

,BenignTraffic,DDoS,DoS,MITM,Mirai,Recon
BenignTraffic,3235,80,155,142,596,127
DDoS,6,88117,2965,0,34,4
DoS,268,8954,22399,5,80,64
MITM,792,13,53,181,210,47
Mirai,206,119,159,21,9719,10
Recon,539,99,190,15,32,381


In [34]:
import numpy as np
test_label_dist = np.unique(y_test, return_counts=True)[1]
test_label_dist

array([ 4335, 91126, 31770,  1296, 10234,  1256])

## Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=100, random_state=0)
clf_rf.fit(x_train, y_train)

RandomForestClassifier(random_state=0)

In [31]:
y_pred = clf_rf.predict(x_train)
generate_metric_report(y_train, y_pred)

,DDoS,DoS,accuracy,macro avg,weighted avg
precision,99.79,99.86,99.8,99.82,99.8
recall,99.97,99.12,99.8,99.54,99.8
f1-score,99.88,99.49,99.8,99.68,99.8


In [47]:
generate_confusion_matrix(y_test, y_pred)

,BenignTraffic,DDoS,DoS,MITM,Mirai,Recon
BenignTraffic,4167,1,11,42,2,112
DDoS,7,88788,2325,1,3,2
DoS,49,5185,26510,13,7,6
MITM,282,2,9,908,7,88
Mirai,26,3,15,20,10165,5
Recon,384,2,24,26,1,819


## XGB Classifier

In [18]:
#gamma=0, learning_rate=0.03, min_child_weight=7, reg_lambda=0.4, subsample=0.6
import xgboost as xgb

initial_model = xgb.XGBClassifier(n_estimators=1000,
                                  gamma=0,
                                  max_depth=None,
                                  learning_rate=0.01,
                                  num_parallel_tree=10,
                                  subsample=0.7,
                                  colsample_bytree=0.8,
                                  colsample_bylevel=0.8,
                                  base_score=0.5,
                    tree_method= 'hist',device = 'cuda',
                    min_child_weight = 7, reg_lambda =0.1,
                    random_state=42, seed=42
                                  )

x_train = df[ ( df['label'] == 'BenignTraffic') | (df['label'] == 'Recon' ) ]
y_train = x_train['label']
x_train = x_train.drop(columns=['label'])
y_train = label_encoder.transform(y_train)
x_train.shape

(28420, 21)

In [7]:
init_mod = initial_model.fit(x_train, y_train)

In [8]:
y_pred = init_mod.predict(x_test)
generate_metric_report(y_test, y_pred)

NameError: name 'x_test' is not defined

In [24]:
generate_confusion_matrix(y_test, y_pred)

ValueError: Length mismatch: Expected axis has 2 elements, new values have 6 elements

## Ensemble

In [7]:
from sklearn.ensemble import VotingClassifier
from cuml.neighbors import KNeighborsClassifier
from cuml.ensemble import RandomForestClassifier

estimators = [
    ("xgb", initial_model),
    ("knn", KNeighborsClassifier(n_neighbors=11)),
    ("rf", RandomForestClassifier(n_estimators=100, random_state=0))
]

voting = VotingClassifier(
    n_jobs=-1,
    estimators=estimators,
    voting = "soft"
)

voting.fit(X, y)

[I] [11:29:36.796086] Unused keyword parameter: n_jobs during cuML estimator initialization


/home/maro/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/maro/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/maro/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=0.5, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.8,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device='cuda',
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None, min_child_weight=7,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=10,
                                            random_state=42, ...)),
                             ('knn', KNeighborsClassifier()),
                             ('rf', RandomForestClassifier())],
                 n_jobs=-1, voting='soft')

In [15]:
y_pred = voting.predict(X)

In [16]:
generate_metric_report(y, y_pred)

,BenignTraffic,DDoS,DoS,MITM,Mirai,Recon,accuracy,macro avg,weighted avg
precision,84.16,91.21,95.02,95.89,99.86,89.36,92.26,92.58,92.51
recall,98.32,98.76,72.44,68.38,99.32,66.08,92.26,83.88,92.26
f1-score,90.69,94.84,82.21,79.83,99.59,75.97,92.26,87.19,91.89


In [39]:
generate_confusion_matrix(y, y_pred)

,BenignTraffic,DDoS,DoS,MITM,Mirai,Recon
BenignTraffic,21615,13,102,49,21,185
DDoS,20,449940,5615,0,4,3
DoS,288,43297,114816,51,10,34
MITM,1598,17,87,4317,21,273
Mirai,186,15,84,54,50923,11
Recon,1975,24,135,31,17,4250


# Submission

In [8]:
# choose your model
final_model = voting
submit = False

In [20]:
testing_data = pd.read_csv(os.path.join(current_path, "data", "raw", "test.csv"))

id = testing_data['Id']
testing_data = testing_data[df.columns[:-1]]
testing_data


,flow_time,header_size,packet_duration,overall_rate,src_rate,dst_rate,fin_packets,urg_packets,rst_packets,max_value,...,fin_flags,syn_flags,rst_flags,psh_flags,ack_flags,protocol_http,protocol_https,protocol_tcp,protocol_udp,protocol_icmp
0,0.000000,54.0,64.00,249.534700,249.534700,0.0,0.0,0.0,0.0,54.0,...,0,0,0,0,0,0,0,1,0,0
1,4.466080,108.0,64.00,0.447820,0.447820,0.0,0.0,0.0,0.0,54.0,...,0,1,0,0,0,0,0,1,0,0
2,0.000000,54.0,64.00,1.295361,1.295361,0.0,0.0,1.0,1.0,54.0,...,0,0,0,1,1,0,0,1,0,0
3,0.000000,0.0,64.00,2.249081,2.249081,0.0,0.0,0.0,0.0,42.0,...,0,0,0,0,0,0,0,0,0,1
4,4.413071,108.0,64.00,0.453199,0.453199,0.0,0.0,0.0,0.0,54.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104282,0.000000,54.0,64.00,10.863896,10.863896,0.0,0.0,0.0,0.0,54.0,...,0,0,0,0,0,1,0,1,0,0
104283,0.000000,54.0,64.00,0.657968,0.657968,0.0,0.0,0.0,0.0,54.0,...,0,0,0,0,0,0,0,1,0,0
104284,0.000000,54.0,64.00,1.356930,1.356930,0.0,1.0,0.0,0.0,54.0,...,1,0,1,0,0,0,0,1,0,0
104285,0.000000,0.0,64.00,23.900780,23.900780,0.0,0.0,0.0,0.0,42.0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
y_pred = voting.predict(testing_data)
# !kaggle competitions submit -c csai-253-project-phase-2 -f submission.csv -m "Message"

In [22]:
# Adding the other prediction layer
testing_data['label'] = y_pred
testing_data_SUS = testing_data[ ( testing_data['label'] == 5) | (testing_data['label'] == 0) ]
sus = testing_data_SUS['label']
x_test2 = testing_data_SUS.drop(columns=['label'])
testing_data['label'].value_counts()

label
1    82214
2    12328
4     5808
0     2927
5      513
3      497
Name: count, dtype: int64

In [23]:
print(np.unique(sus, return_counts=True))
sus = clf_rf.predict(x_test2)
print(np.unique(sus, return_counts=True))
mask = (testing_data['label'] ==5) | (testing_data['label'] == 0)
testing_data.loc[mask,['label']] = sus+1
testing_data['label'].value_counts()

(array([0, 5]), array([2927,  513]))
(array([0, 5]), array([2829,  611]))


label
1    85043
2    12328
4     5808
6      611
3      497
Name: count, dtype: int64

In [39]:
result = pd.DataFrame(label_encoder.inverse_transform(testing_data['label']), columns=['Target'])
result['Id'] = id
result = result[['Id', 'Target']]

# correct_names = ['DDoS', 'Recon', 'BenignTraffic', 'MITM', 'DoS', 'Mirai']
# result['Target'] = result['Target'].map({name: name for name in correct_names})

result.to_csv(os.path.join(current_path, "data", "submission.csv"), index=False)
print(result['Target'].value_counts())

Target
DDoS             78152
DoS              16390
Mirai             5808
BenignTraffic     2927
Recon              513
MITM               497
Name: count, dtype: int64


In [14]:
# Save trained ensemble model
joblib.dump(voting, 'voting_model_XGB_subst.pkl.pkl')


['voting_model_XGB_subst.pkl.pkl']

# Loading previous models

In [4]:
# Loading the model from previously saved file
import joblib
voting = joblib.load('voting_model_XGB_subst.pkl.pkl')

In [5]:
voting

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=0.5, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.8,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device='cuda',
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None, min_child_weight=7,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=10,
                                            random_state=42, ...)),
                             ('knn', KNeighborsClassifier()),
                             ('rf', RandomForestClassifier())],
                 n_jobs=-1, voting='soft')